In [38]:
from __future__ import print_function
# key libs
import numpy as np
import re
import nltk
import pandas as pd
import glob
import codecs

# nlp libs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 

# processing
from sklearn.model_selection import train_test_split

# LDA
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF


# bring in my pickled vectorizers
import pickle
import dill
# import import_ipynb
# from process_tokenize import my_tokenizer

In [39]:
# what are we interestd in
# set this up to read the correct set of files we are interested in

n_gram = '1gm' # options - 1gm , 2gm
stem_type = 'lemma' # options - lemma,snow

In [40]:
# set random state
RSEED = 0

### 1. OPEN - TERM FREQUENCY FILES 
i.e. output of CountVectorizer

In [41]:
tf_train = pd.read_csv('../data/vectors/cv_'+ n_gram +'_'+ stem_type +'_train.csv', compression = 'gzip')
tf_train = tf_train.drop(columns='Unnamed: 0')

tf_test = pd.read_csv('../data/vectors/cv_'+ n_gram +'_'+ stem_type +'_test.csv', compression = 'gzip')
tf_test = tf_test.drop(columns='Unnamed: 0')

In [42]:
tf_train.head()

,book_title,author_name,book_location,aaron,aback,abandon,abandoned,abandoning,abandonment,abasement,...,zealand,zealous,zealously,zenith,zephyr,zero,zest,zigzag,zone,zoological
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0,0,2,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,3,0,0


In [43]:
# load the appropriate tf_vectorizer
tf_vectorizer = dill.load(open('../data/vectors/tf_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))

### 2. OPEN - TERM FREQUENCY INVERSE DOCUMENT FREQUENCY FILES
i.e output of TfidfVectorizer 

In [9]:
tfid_train = pd.read_csv('../data/vectors/tfid_'+ n_gram +'_'+ stem_type +'_train.csv',compression = 'gzip')
tfid_train = tfid_train.drop(columns='Unnamed: 0')

tfid_test = pd.read_csv('../data/vectors/tfid_'+ n_gram +'_'+ stem_type +'_test.csv', compression = 'gzip')
tfid_test = tfid_test.drop(columns='Unnamed: 0')

In [10]:
tfid_train.head()

,book_title,author_name,book_location,aaron,aback,abandon,abandoned,abandoning,abandonment,abasement,...,zealand,zealous,zealously,zenith,zephyr,zero,zest,zigzag,zone,zoological
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.000000
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.0,0.0,0.004126,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.005299
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.0,0.0,0.006395,0.002774,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00448,0.00000,0.0,0.000000
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.005872,...,0.0,0.0,0.0,0.0,0.0,0.00554,0.00000,0.00000,0.0,0.000000
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.0,0.0,0.000000,0.013355,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.03942,0.0,0.000000


In [11]:
# load the appropriate tf_vectorizer
#tfid_vectorizer = pickle.load(open('../data/vectors/tfid_vectorizer_'+ stem_type +'_'+ n_gram +'.pkl', 'rb'))
tfid_vectorizer = dill.load(open('../data/vectors/tfid_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))

### 3. HELPER FUNCTIONS

In [12]:
#function to display topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print (f'Topic: {topic_idx}')
        print (" , ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [13]:
#function to display books for a topic
def find_topic_books(df,number):
    topics = list(df.columns)[3:]
    #print(topics)
    for t in topics:
        temp = df.sort_values(by=t,ascending=False)
        print('\n------'+t+'-------') 
        print(temp.iloc[0:number+1,0:2])    

### 4. DIMENSIONALITY REDUCTION - TOPIC MODELLING
1. LDA
2. NMF
3. LSA / LSI ? - These are rather old and don't give good results


use the Term Frequency matrix as Term Frequency Inverse Document Frequency does not work well

In [28]:
# set how many topics you want
topic_number = 15

### A. LDA with TF
LatentDirichletAllocation with the Term Frequency Allocation
Use the Term Frequency matrix as Term Frequency Inverse Document Frequency does not work well
LDA can only use raw term counts for LDA because it is a probabilistic graphical model

In [44]:
# Run LDA 
lda_tf = LatentDirichletAllocation(learning_method= 'batch',n_components=topic_number, random_state=RSEED, batch_size=128,n_jobs=-1)
lda_tf.fit(tf_train.iloc[:,3:])

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=15, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [45]:
# transform vectors
vec_lda_train = lda_tf.transform(tf_train.iloc[:,3:])
vec_lda_test = lda_tf.transform(tf_test.iloc[:,3:])

In [46]:
dill.dump(lda_tf, open('../data/vectors/lda_'+ stem_type + '_' + n_gram,'wb'))

In [47]:
print('------PERPLEXITY------')
print(lda_tf.perplexity(tf_train.iloc[:,3:]))
print(lda_tf.perplexity(tf_test.iloc[:,3:]))

------PERPLEXITY------
6417.809587077815
6914.7525078331455


In [54]:
# get the topics and words for the topics in lda
no_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
display_topics(lda_tf, tf_feature_names, no_top_words)

Topic: 0
mr , george , duke , harry , mary , james , john , captain , charles , colonel
Topic: 1
captain , boat , deck , vessel , island , sail , mate , shore , sailor , crew
Topic: 2
peter , mr , didn , jimmy , joe , tommy , ain , alice , bob , couldn
Topic: 3
government , principle , nation , political , religion , trade , moral , united , parliament , liberty
Topic: 4
thou , thy , thee , poet , poem , footnote , christ , sin , greek , ti
Topic: 5
dick , tom , camp , jim , sam , trail , snow , gun , didn , rover
Topic: 6
specie , plant , colour , variety , male , surface , female , science , distinct , period
Topic: 7
american , literature , reader , literary , author , artist , poet , modern , novel , century
Topic: 8
smiled , didn , remarked , mr , glanced , isn , presently , grey , nodded , monsieur
Topic: 9
indian , island , lake , forest , native , savage , village , snow , canoe , shore
Topic: 10
thou , prince , thee , queen , thy , knight , sword , castle , nay , richard
Topic

In [55]:
# add back details
lda_train = pd.DataFrame(vec_lda_train, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
lda_train.insert(loc=0, column='book_location', value=tf_train['book_location'])
lda_train.insert(loc=0, column='author_name', value=tf_train['author_name'])
lda_train.insert(loc=0, column='book_title', value=tf_train['book_title'])
lda_train.to_csv('../data/vectors/lda_'+ n_gram+'_'+ stem_type +'_train.csv')
lda_train.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.024503,0.000014,0.000014,0.019456,0.004147,0.000014,0.000014,0.188634,0.199965,0.000014,0.000014,0.000014,0.000014,0.289327,0.273858
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.122082,0.000007,0.303533,0.012188,0.000007,0.000007,0.013001,0.066539,0.122667,0.000007,0.000007,0.000007,0.123463,0.236478,0.000007
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.201496,0.087346,0.029985,0.027774,0.000010,0.022922,0.002199,0.137414,0.341142,0.018526,0.009753,0.052020,0.000010,0.068418,0.000984
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.000020,0.000020,0.007362,0.000020,0.000020,0.281786,0.000020,0.000020,0.264266,0.152951,0.000020,0.000020,0.086513,0.206946,0.000020
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.000020,0.069924,0.030690,0.036002,0.000020,0.000020,0.059825,0.018630,0.122308,0.000020,0.031033,0.000020,0.398182,0.003914,0.229392


In [56]:
# add the book details and write to file for test
lda_test = pd.DataFrame(vec_lda_test, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
lda_test.insert(loc=0, column='book_location', value=tf_test['book_location'])
lda_test.insert(loc=0, column='author_name', value=tf_test['author_name'])
lda_test.insert(loc=0, column='book_title', value=tf_test['book_title'])
lda_test.to_csv('../data/vectors/lda_'+ n_gram+'_'+ stem_type +'_test.csv')
lda_test.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15
0,Great Catherine,George Bernard Shaw,../data/gutenberg/George Bernard Shaw___Great ...,0.101817,0.000024,0.000863,0.000024,0.000999,0.000024,0.000024,0.215072,0.055150,0.000024,0.089095,0.102721,0.218641,0.130368,0.085156
1,Short Cruises,William Wymark Jacobs,../data/gutenberg/William Wymark Jacobs___Shor...,0.016490,0.108463,0.229974,0.000008,0.000008,0.157654,0.000008,0.000008,0.020388,0.000008,0.000008,0.000008,0.092946,0.374023,0.000008
2,Richard Dare's Venture,Edward Stratemeyer,../data/gutenberg/Edward Stratemeyer___Richard...,0.113744,0.067719,0.177472,0.009760,0.000010,0.093262,0.000010,0.000010,0.007516,0.000010,0.136102,0.016032,0.003753,0.374590,0.000010
3,William Harvey And The Discovery Of The Circul...,Thomas Henry Huxley,../data/gutenberg/Thomas Henry Huxley___Willia...,0.000050,0.000050,0.000050,0.238633,0.009119,0.000050,0.678779,0.072914,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050,0.000050
4,Beyond,John Galsworthy,../data/gutenberg/John Galsworthy___Beyond.txt,0.011050,0.000004,0.132279,0.000004,0.018823,0.000004,0.000004,0.015262,0.620572,0.000004,0.000004,0.000004,0.113574,0.056881,0.031530


In [57]:
# find documents for topic
no_documents = 10
find_topic_books(lda_train,no_documents)


------topic_1-------
                              book_title       author_name
1196               The Journal to Stella    Jonathan Swift
2011                           Lady Anna  Anthony Trollope
1716  Sir Harry Hotspur of Humblethwaite  Anthony Trollope
1470            Mr. Scarborough's Family  Anthony Trollope
682                      Is He Popenjoy?  Anthony Trollope
503          The Golden Lion of Granpere  Anthony Trollope
595                         Cousin Henry  Anthony Trollope
464                           Marion Fay  Anthony Trollope
120                Trial of Duncan Terig  Sir Walter Scott
289                 The American Senator  Anthony Trollope
66                   The Duke's Children  Anthony Trollope

------topic_2-------
                            book_title            author_name
1472                The Life of a Ship         R M Ballantyne
2153                  Jarwin and Cuffy         R M Ballantyne
123                     The Lighthouse         R M Ballantyne


In [52]:
# setup for visualization
vec_tf=csr_matrix(tf_train.iloc[:,3:].values)
pyLDAvis.sklearn.prepare(lda_tf,vec_tf,tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      10.099042        1       1  0.083307  0.004030
12      9.504929        1       2  0.011437 -0.083111
3       9.043578        1       3 -0.218099 -0.048251
10      8.576013        1       4 -0.002441 -0.066635
7       7.683422        1       5 -0.083790 -0.046552
4       7.172660        1       6 -0.081922 -0.095986
0       7.069231        1       7 -0.010395 -0.144722
11      5.996171        1       8 -0.016040  0.059144
1       5.752567        1       9  0.054394  0.082319
6       5.449676        1      10 -0.219937  0.172510
13      5.150987        1      11  0.145401 -0.041787
5       5.119750        1      12  0.177471  0.112496
14      5.087141        1      13  0.000299  0.010228
9       4.693822        1      14 -0.004996  0.100341
2       3.601011        1      15  0.165310 -0.014022, topic_info=      Category           Freq          Term          Total  loglift  logprob
term                                                                        
10094  Default  121626.000000            mr  121626.000000  30.0000  30.0000
15651  Default   45205.000000          thou   45205.000000  29.0000  29.0000
2202   Default   57146.000000       captain   57146.000000  28.0000  28.0000
15720  Default   34732.000000           thy   34732.000000  27.0000  27.0000
15585  Default   29155.000000          thee   29155.000000  26.0000  26.0000
1694   Default   38466.000000          boat   38466.000000  25.0000  25.0000
14501  Default   19018.000000        specie   19018.000000  24.0000  24.0000
4320   Default   37646.000000          didn   37646.000000  23.0000  23.0000
15822  Default   26783.000000           tom   26783.000000  22.0000  22.0000
828    Default   26845.000000          army   26845.000000  21.0000  21.0000
4308   Default   19706.000000          dick   19706.000000  20.0000  20.0000
8064   Default   24661.000000        indian   24661.000000  19.0000  19.0000
11262  Default   19582.000000         peter   19582.000000  18.0000  18.0000
10564  Default   28090.000000       officer   28090.000000  17.0000  17.0000
11429  Default   17040.000000         plant   17040.000000  16.0000  16.0000
16065  Default   15970.000000         troop   15970.000000  15.0000  15.0000
16273  Default   29101.000000         uncle   29101.000000  14.0000  14.0000
1060   Default   21618.000000          aunt   21618.000000  13.0000  13.0000
8588   Default   27093.000000        island   27093.000000  12.0000  12.0000
6719   Default   29428.000000        george   29428.000000  11.0000  11.0000
6889   Default   24012.000000    government   24012.000000  10.0000  10.0000
3900   Default   14611.000000          deck   14611.000000   9.0000   9.0000
2148   Default   19395.000000          camp   19395.000000   8.0000   8.0000
2808   Default   23727.000000       colonel   23727.000000   7.0000   7.0000
14374  Default   24935.000000       soldier   24935.000000   6.0000   6.0000
4916   Default   19770.000000          duke   19770.000000   5.0000   5.0000
7248   Default   17199.000000         harry   17199.000000   4.0000   4.0000
11520  Default   17853.000000          poet   17853.000000   3.0000   3.0000
11518  Default   13960.000000          poem   13960.000000   2.0000   2.0000
9594   Default   23462.000000          mary   23462.000000   1.0000   1.0000
...        ...            ...           ...            ...      ...      ...
2468   Topic15    1086.320796        cherry    1983.147998   2.7221  -6.8839
8692   Topic15    4595.146152           joe    8732.897307   2.6819  -5.4417
14134  Topic15     484.386742          skip     885.234189   2.7210  -7.6916
1698   Topic15    3542.921984           bob    7247.581660   2.6082  -5.7018
14651  Topic15    1041.865688      squirrel    2009.095673   2.6673  -6.9257
17556  Topic15    2109.817063           yer    4313.325312   2.6088  -6.2201
349    Topic15    1140.671543       

### B. NMF WITH TFID
NMF workd better with TFID

In [29]:
# Run NMF
nmf_tfid = NMF(n_components=topic_number, random_state=RSEED, max_iter=500)
nmf_tfid.fit(tfid_train.iloc[:,3:])

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=500,
  n_components=15, random_state=0, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [30]:
# transform vectors
vec_nmf_train = nmf_tfid.transform(tfid_train.iloc[:,3:])
vec_nmf_test = nmf_tfid.transform(tfid_test.iloc[:,3:])

In [58]:
dill.dump(nmf_tfid, open('../data/vectors/nmf_'+ stem_type + '_' + n_gram,'wb'))

In [32]:
no_top_words = 10
tfid_feature_names = tfid_vectorizer.get_feature_names()
display_topics(nmf_tfid, tfid_feature_names, no_top_words)

Topic: 0
girl , answered , cried , didn , big , replied , lip , slowly , road , smile
Topic: 1
thou , thy , thee , hath , ti , thine , hast , soul , sweet , heaven
Topic: 2
government , public , nation , war , political , principle , opinion , class , society , constitution
Topic: 3
mr , miss , girl , didn , husband , aunt , couldn , isn , wouldn , won
Topic: 4
sir , lord , gentleman , john , miss , george , harry , master , madame , london
Topic: 5
s , wot , arter , ave , ginger , sam , bob , agin , ead , skipper
Topic: 6
captain , ship , boat , deck , island , vessel , shore , sail , board , crew
Topic: 7
dick , tom , sam , rover , baxter , dan , fred , cried , answered , didn
Topic: 8
art , poet , poem , literature , literary , english , author , poetry , character , artist
Topic: 9
god , christ , lord , sin , jesus , soul , heaven , christian , church , holy
Topic: 10
specie , animal , plant , darwin , science , hypothesis , bird , natural , condition , organic
Topic: 11
officer , 

In [33]:
# add back details
nmf_train = pd.DataFrame(vec_nmf_train, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
nmf_train.insert(loc=0, column='book_location', value=tfid_train['book_location'])
nmf_train.insert(loc=0, column='author_name', value=tfid_train['author_name'])
nmf_train.insert(loc=0, column='book_title', value=tfid_train['book_title'])
nmf_train.to_csv('../data/vectors/nmf_'+ n_gram+'_'+ stem_type +'_train.csv')
nmf_train.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.049734,0.000000,0.013313,0.031046,0.094106,0.000042,0.000000,0.003954,0.033866,0.000000,0.00000,0.000000,0.024112,0.016215,0.037175
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.002097,0.000000,0.008095,0.072340,0.030825,0.000000,0.000000,0.006800,0.008982,0.002681,0.00000,0.000000,0.247304,0.000047,0.000000
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.058647,0.000000,0.011558,0.012376,0.108296,0.000000,0.013953,0.003363,0.012292,0.002835,0.00000,0.004707,0.015935,0.000000,0.000000
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.046908,0.000477,0.000000,0.021095,0.000000,0.000000,0.003144,0.005293,0.000000,0.002739,0.00000,0.023262,0.000000,0.000000,0.008548
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.054830,0.001287,0.024175,0.011042,0.125689,0.000000,0.008057,0.000000,0.005426,0.005772,0.03009,0.000000,0.000000,0.000000,0.005405


In [34]:
# add back details
nmf_test = pd.DataFrame(vec_nmf_test, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
nmf_test.insert(loc=0, column='book_location', value=tfid_test['book_location'])
nmf_test.insert(loc=0, column='author_name', value=tfid_test['author_name'])
nmf_test.insert(loc=0, column='book_title', value=tfid_test['book_title'])
nmf_test.to_csv('../data/vectors/nmf_'+ n_gram+'_'+ stem_type +'_test.csv')
nmf_test.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15
0,Great Catherine,George Bernard Shaw,../data/gutenberg/George Bernard Shaw___Great ...,0.005446,0.000000,0.002012,0.006462,0.022827,0.000574,0.010053,0.000253,0.017388,0.011574,0.00000,0.008527,0.004229,0.036841,0.000000
1,Short Cruises,William Wymark Jacobs,../data/gutenberg/William Wymark Jacobs___Shor...,0.014138,0.000000,0.000000,0.061661,0.010012,0.397639,0.004839,0.008816,0.000000,0.000000,0.00000,0.000000,0.002227,0.000000,0.009618
2,Richard Dare's Venture,Edward Stratemeyer,../data/gutenberg/Edward Stratemeyer___Richard...,0.021555,0.000000,0.000000,0.018932,0.049229,0.000000,0.001623,0.029185,0.000000,0.000000,0.00000,0.000000,0.005849,0.026485,0.000000
3,William Harvey And The Discovery Of The Circul...,Thomas Henry Huxley,../data/gutenberg/Thomas Henry Huxley___Willia...,0.000000,0.000000,0.015286,0.000000,0.003093,0.000000,0.001815,0.004591,0.009108,0.008289,0.06898,0.001762,0.000000,0.004470,0.000000
4,Beyond,John Galsworthy,../data/gutenberg/John Galsworthy___Beyond.txt,0.153693,0.004038,0.000000,0.046356,0.025651,0.000000,0.000000,0.000000,0.006931,0.004107,0.00000,0.000000,0.000000,0.000000,0.021348


In [36]:
# find documents for topic
no_documents = 30
find_topic_books(nmf_train,no_documents)


------topic_1-------
                                       book_title                author_name
205                                 Women in Love               D H Lawrence
1991                                   Five Tales            John Galsworthy
674   The Country of the Blind, And Other Stories       Herbert George Wells
2189                              Tales of Unrest              Joseph Conrad
280                              Frontier Stories                 Bret Harte
2106                                  The Rainbow               D H Lawrence
1327                              The Dark Flower            John Galsworthy
69                 Villa Rubein and Other Stories            John Galsworthy
642                                   Novel Notes       Jerome Klapka Jerome
2023                    An Outcast of the Islands              Joseph Conrad
2340       The Door in the Wall And Other Stories       Herbert George Wells
1003                       In the Carquinez Woods     

In [37]:
# setup for visualization
vec_tfid=csr_matrix(tfid_train.iloc[:,3:].values)
pyLDAvis.sklearn.prepare(nmf_tfid,vec_tfid,tfid_vectorizer)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      13.716801        1       1  0.062522  0.017889
4      11.732522        1       2 -0.053996 -0.101035
2       9.586207        1       3 -0.198540 -0.177444
8       8.543531        1       4 -0.174431 -0.025405
11      8.096507        1       5 -0.046163  0.015262
3       8.014518        1       6  0.058205 -0.184750
13      6.327428        1       7 -0.103962  0.086726
10      6.107456        1       8 -0.180100 -0.078271
9       5.703418        1       9 -0.169842  0.075905
1       5.672379        1      10 -0.085182  0.237760
6       5.663605        1      11  0.028193 -0.007079
14      4.299792        1      12  0.088627  0.245974
7       2.862034        1      13  0.228099 -0.047390
12      2.748798        1      14  0.252288  0.009378
5       0.925002        1      15  0.294280 -0.067520, topic_info=      Category        Freq          Term       Total  loglift  logprob
term                                                                  
10481  Default  179.000000            mr  179.000000  30.0000  30.0000
13849  Default   80.000000             s   80.000000  29.0000  29.0000
4455   Default   95.000000          dick   95.000000  28.0000  28.0000
7110   Default  126.000000           god  126.000000  27.0000  27.0000
11694  Default   76.000000         peter   76.000000  26.0000  26.0000
9156   Default  117.000000          king  117.000000  25.0000  25.0000
16456  Default   78.000000           tom   78.000000  24.0000  24.0000
8090   Default   75.000000  illustration   75.000000  23.0000  23.0000
13915  Default   62.000000           sam   62.000000  22.0000  22.0000
16281  Default   72.000000          thou   72.000000  21.0000  21.0000
16353  Default   64.000000           thy   64.000000  20.0000  20.0000
9002   Default   44.000000         jimmy   44.000000  19.0000  19.0000
10273  Default   86.000000          miss   86.000000  18.0000  18.0000
16214  Default   49.000000          thee   49.000000  17.0000  17.0000
2283   Default   70.000000       captain   70.000000  16.0000  16.0000
12316  Default   54.000000        prince   54.000000  15.0000  15.0000
13766  Default   36.000000         rover   36.000000  14.0000  14.0000
4467   Default   63.000000          didn   63.000000  13.0000  13.0000
9658   Default   88.000000          lord   88.000000  12.0000  12.0000
18131  Default   28.000000           wot   28.000000  11.0000  11.0000
14656  Default   95.000000           sir   95.000000  10.0000  10.0000
2628   Default   40.000000        christ   40.000000   9.0000   9.0000
14443  Default   51.000000          ship   51.000000   8.0000   8.0000
1596   Default   28.000000         billy   28.000000   7.0000   7.0000
7021   Default   69.000000          girl   69.000000   6.0000   6.0000
1752   Default   47.000000          boat   47.000000   5.0000   5.0000
1133   Default   20.000000           ave   20.000000   4.0000   4.0000
6978   Default   39.000000        george   39.000000   3.0000   3.0000
1756   Default   23.000000           bob   23.000000   2.0000   2.0000
885    Default   17.000000         arter   17.000000   1.0000   1.0000
...        ...         ...           ...         ...      ...      ...
11120  Topic15    2.409952          ouse    4.199471   4.1278  -5.5134
3255   Topic15    1.603056      conjurer    2.840226   4.1112  -5.9211
820    Topic15    2.592884           ard    4.683307   4.0919  -5.4402
12038  Topic15    3.221953          pore    5.846557   4.0873  -5.2230
8991   Topic15    1.064239         jetty    2.043316   4.0308  -6.3307
6863   Topic15    3.705994           gal    7.251346   4.0119  -5.0830
11803  Topic15    2.438640          pint    4.863660   3.9928  -5.5015
10280  Topic15    3.050967        missis    6.105255   3.9894  -5.2775
11021  Topic15    1.037676           ope    2.115469   3.9708  -6.3560
17767  Topic15    2.079868      watchman    4.308803   3.